### Loading Data
Dan Constantini, Tom Hayat et Alexandre Attia
This script loads 11 data class from imagenet and sort the files according to their name

In [1]:
require 'torch'
require 'xlua'
require 'image'
require 'nn'
require 'optim'
require 'cunn'


--classes={'bridge', 'building', 'city', 'eiffel_tower','elephant', 'landscape', 'lion', 'monkey', 'people', 'tower', 'water'}
total = 0
for dir in paths.iterdirs("./dataset/train/") do
    files = {}
    for file in paths.files(paths.concat('./dataset/train/', dir)) do
       if file:find('JPEG' .. '$') then
          table.insert(files, 1)
       end
    end
   
    if #files == 0 then
       error('given directory doesnt contain any files of type: ')
    end

    total = total + #files
    collectgarbage()
end

print('total', total)

imagesAll = torch.Tensor(total,3,64,64) 
labelsAll = torch.Tensor(total)
compteur = 0
NumberOfImages = {0}
for dir in paths.iterdirs("./dataset/train/") do
    compteur = compteur +1
    files = {}
    for file in paths.files(paths.concat('./dataset/train/', dir)) do
       if file:find('JPEG' .. '$') then
          table.insert(files, paths.concat(paths.concat('./dataset/train/',dir), file))
       end
    end
   
    if #files == 0 then
       error('given directory doesnt contain any files of type: ')
    end

    lastElement = NumberOfImages[#NumberOfImages]
    table.insert(NumberOfImages, #files + lastElement)
    for i=1,(#files) do
        temp = image.load(files[i])
        temp2 = torch.Tensor(3,64,64) 
        if (temp:size()[1]==1) then
            temp2[{{1},{},{}}] = temp
            temp2[{{2},{},{}}] = temp
            temp2[{{3},{},{}}] = temp
        else 
            temp2 = temp
        end
        imagesAll[i+lastElement] = temp2 
        labelsAll[i+lastElement] = compteur
    end
    collectgarbage()
end


-- Nombre d'images courleurs: 1683 
trainData = {
    data = torch.Tensor(total, 3, 64, 64),
    labels = torch.Tensor(total),
    size = function() return total end,

}
--[[create test set:
testData = {
      data = torch.Tensor(tesize, 1, 32, 32),
      labels = torch.Tensor(tesize),
      size = function() return tesize end
   }
]]--

for i=1,total do
   trainData.data[i] = imagesAll[i]
   trainData.labels[i] = labelsAll[i]
end
--for i=trsize+1,tesize+trsize do
   --testData.data[i-trsize] = imagesAll[labelsShuffle[i]][1]:clone()
   --testData.labels[i-trsize] = labelsAll[labelsShuffle[i]]
--end

total	1861	


In [2]:
trainData

{
  data : DoubleTensor - size: 1861x3x64x64
  size : function: 0x0b1d6bd8
  labels : DoubleTensor - size: 1861
}


In [3]:
--for i=1,total do 
    --itorch.image(imagesAll[i])
    --print(labelsAll[i])
--end

## Preprocessing Data

In [4]:
-- Convert all images to YUV
collectgarbage()
print '==> preprocessing data: colorspace RGB -> YUV'
for i = 1,total do
   trainData.data[i] = image.rgb2yuv(trainData.data[i])
end
--for i = 1,325 do
  -- testData.data[i] = image.rgb2yuv(testData.data[i])
--end
-- Name channels for convenience
channels = {'y','u','v'}

-- Normalize each channel, and store mean/std
-- per channel. These values are important, as they are part of
-- the trainable parameters. At test time, test data will be normalized
-- using these values.
print '==> preprocessing data: normalize each feature (channel) globally'
mean = {}
std = {}
for i,channel in ipairs(channels) do
   -- normalize each channel globally:
   mean[i] = trainData.data[{ {},i,{},{} }]:mean()
   std[i] = trainData.data[{ {},i,{},{} }]:std()
   trainData.data[{ {},i,{},{} }]:add(-mean[i])
   trainData.data[{ {},i,{},{} }]:div(std[i])
end

-- Normalize test data, using the training means/stds
--for i,channel in ipairs(channels) do
   -- normalize each channel globally:
   --testData.data[{ {},i,{},{} }]:add(-mean[i])
   --testData.data[{ {},i,{},{} }]:div(std[i])
--end
-- Local normalization
print '==> preprocessing data: normalize all three channels locally'

-- Define the normalization neighborhood:
neighborhood = image.gaussian1D(13)

-- Define our local normalization operator (It is an actual nn module, 
-- which could be inserted into a trainable model):
normalization = nn.SpatialContrastiveNormalization(1, neighborhood)

-- Normalize all channels locally:
for c in ipairs(channels) do
   for i = 1,total do
      trainData.data[{ i,{c},{},{} }] = normalization:forward(trainData.data[{ i,{c},{},{} }])
   end
   --for i = 1,325 do
      --testData.data[{ i,{c},{},{} }] = normalization:forward(testData.data[{ i,{c},{},{} }])
   --end
end

print '==> verify statistics'
for i,channel in ipairs(channels) do
   trainMean = trainData.data[{ {},i }]:mean()
   trainStd = trainData.data[{ {},i }]:std()

   --testMean = testData.data[{ {},i }]:mean()
   --testStd = testData.data[{ {},i }]:std()

   print('training data, '..channel..'-channel, mean: ' .. trainMean)
   print('training data, '..channel..'-channel, standard deviation: ' .. trainStd)

   --print('test data, '..channel..'-channel, mean: ' .. testMean)
   --print('test data, '..channel..'-channel, standard deviation: ' .. testStd)
end


==> preprocessing data: colorspace RGB -> YUV	


==> preprocessing data: normalize each feature (channel) globally	


==> preprocessing data: normalize all three channels locally	


==> verify statistics	


training data, y-channel, mean: -0.01881966926504	
training data, y-channel, standard deviation: 0.86958206694476	


training data, u-channel, mean: 0.01750703263677	
training data, u-channel, standard deviation: 0.8532538826994	


training data, v-channel, mean: -0.015721198933372	
training data, v-channel, standard deviation: 0.84829254696119	


## Model

In [5]:
-- define model to train
classes={'bridge', 'building', 'city', 'eiffel_tower','elephant', 'landscape', 'lion', 'monkey', 'people', 'tower', 'water'}
-- define model to train

model = nn.Sequential()
 --stage 1 : mean+std normalization -> filter bank -> squashing -> max pooling
model:add(nn.SpatialConvolutionMM(3,64,5,5))
model:add(nn.ReLU())
model:add(nn.SpatialMaxPooling(2, 2, 2, 2))
-- stage 2 : filter bank -> squashing -> max pooling
model:add(nn.SpatialConvolutionMM(64,64,5,5))
model:add(nn.ReLU())
model:add(nn.SpatialMaxPooling(2, 2, 2, 2))

-- stage 3 : standard 2-layer neural network
model:add(nn.View(64*13*13))
model:add(nn.Linear(64*13*13, #classes))
--model:add(nn.ReLU())
--model:add(nn.Linear(128,#classes))
model:add(nn.LogSoftMax())

print '==> vizualisation du modèle'
print(model:__tostring())
-- retrieve parameters and gradients. this helps us to use the optim package
parameters,gradParameters = model:getParameters()
-- loss function: negative log-likelihood
criterion = nn.ClassNLLCriterion()
print (criterion)
batchSize = 64 -- sets the mini-Batch size
-- this matrix records the current confusion across classes
confusion = optim.ConfusionMatrix(classes)


==> vizualisation du modèle	
nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> output]
  (1): nn.SpatialConvolutionMM(3 -> 64, 5x5)
  (2): nn.ReLU
  (3): nn.SpatialMaxPooling(2x2, 2,2)
  (4): nn.SpatialConvolutionMM(64 -> 64, 5x5)
  (5): nn.ReLU
  (6): nn.SpatialMaxPooling(2x2, 2,2)
  (7): nn.View(10816)
  (8): nn.Linear(10816 -> 11)
  (9): nn.LogSoftMax
}	


nn.ClassNLLCriterion
{
  sizeAverage : true
  output : 0
  gradInput : DoubleTensor - empty
  output_tensor : DoubleTensor - size: 1
  target : LongTensor - size: 1
  total_weight_tensor : DoubleTensor - size: 1
}


## Linear Model

In [6]:
classes={'bridge', 'building', 'city', 'eiffel_tower','elephant', 'landscape', 'lion', 'monkey', 'people', 'tower', 'water'}
-- define model to train

model = nn.Sequential()
model:add(nn.View(3*64*64))
model:add(nn.Linear(3*64*64, #classes))
model:add(nn.LogSoftMax())
print '==> vizualisation du modèle'
print(model:__tostring())
-- retrieve parameters and gradients. this helps us to use the optim package
parameters,gradParameters = model:getParameters()
-- loss function: negative log-likelihood
criterion = nn.ClassNLLCriterion()
print (criterion)
batchSize = 64 -- sets the mini-Batch size
-- this matrix records the current confusion across classes
confusion = optim.ConfusionMatrix(classes)

==> vizualisation du modèle	
nn.Sequential {
  [input -> (1) -> (2) -> (3) -> output]
  (1): nn.View(12288)
  (2): nn.Linear(12288 -> 11)
  (3): nn.LogSoftMax
}	


nn.ClassNLLCriterion
{
  sizeAverage : true
  output : 0
  gradInput : DoubleTensor - empty
  output_tensor : DoubleTensor - size: 1
  target : LongTensor - size: 1
  total_weight_tensor : DoubleTensor - size: 1
}


## Other model, test with CUDA

In [ ]:
classes={'bridge', 'building', 'city', 'eiffel_tower','elephant', 'landscape', 'lion', 'monkey', 'people', 'tower', 'water'}
local noutputs = 11

-- input dimensions: faces!
local nfeats = 3
local width = 64
local height = 64

-- hidden units, filter sizes (for ConvNet only):
local nstates = {32,64}
local filtsize = {5, 7}
local poolsize = 4

local CNN = nn.Sequential()

-- stage 1: conv+max
CNN:add(nn.SpatialConvolutionMM(nfeats, nstates[1], filtsize[1], filtsize[1]))
CNN:add(nn.Threshold())
CNN:add(nn.SpatialMaxPooling(poolsize,poolsize,poolsize,poolsize))

-- stage 2: conv+max
CNN:add(nn.SpatialConvolutionMM(nstates[1], nstates[2], filtsize[2], filtsize[2]))
CNN:add(nn.Threshold())

local classifier = nn.Sequential()
-- stage 3: linear
classifier:add(nn.Reshape(nstates[2]))
classifier:add(nn.Linear(nstates[2], 2))

-- stage 4 : log probabilities
classifier:add(nn.LogSoftMax())
local model = nn.Sequential()
model:add(CNN)
model:add(classifier)

print '==> vizualisation du modèle'
print(model:__tostring())
-- retrieve parameters and gradients. this helps us to use the optim package
parameters,gradParameters = model:getParameters()
-- loss function: negative log-likelihood
criterion = nn.ClassNLLCriterion()
print (criterion)
batchSize = 64 -- sets the mini-Batch size
-- this matrix records the current confusion across classes
confusion = optim.ConfusionMatrix(classes)



In [ ]:
--passage en CUDA
--model = model:cuda()
--criterion = criterion:cuda()
--trainData.data = trainData.data:cuda()
--trainData.labels = trainData.labels:cuda()

In [7]:
-- training function
evaluations={}
times={}
time=0
collectgarbage()
function train(dataset)
   -- epoch tracker
   epoch = epoch or 1

   -- do one epoch
   print('<trainer> on training set:')
   print("<trainer> online epoch # " .. epoch .. ' [batchSize = ' .. batchSize .. ']')
   for t = 1,dataset:size(),batchSize do

      -- create mini batch
      local inputs = {}
      local targets = {}
      for i = t,math.min(t+batchSize-1,dataset:size()) do
         -- load new sample
         local input = dataset.data[i]
         local target = dataset.labels[i]
         table.insert(inputs, input)
         table.insert(targets, target)
      end
      -- create closure to evaluate f(X) and df/dX
      local feval = function(x)
                       -- get new parameters
                       if x ~= parameters then
                          parameters:copy(x)
                       end

                       -- reset gradients
                       gradParameters:zero()

                       -- f is the average of all criterions
                       local f = 0

                       -- evaluate function for complete mini batch
                       for i = 1,#inputs do
                          -- estimate f
                          local output = model:forward(inputs[i])
                          local err = criterion:forward(output, targets[i])
                          f = f + err

                          -- estimate df/dW
                          local df_do = criterion:backward(output, targets[i])
                          model:backward(inputs[i], df_do)

                          -- update confusion
                          confusion:add(output, targets[i])                        
                       end

                       -- normalize gradients and f(X)
                       gradParameters:div(#inputs)
                       f = f/#inputs
                       

                       -- return f and df/dX
                       return f,gradParameters
                    end


      config = config or {learningRate = 1e-3,
              weightDecay = 0,
                momentum = 0,
              learningRateDecay = 5e-7}
      optim.sgd(feval, parameters, config)
      
   end

   -- print confusion matrix
   print(confusion)
  table.insert(evaluations,confusion.averageValid)
   confusion:zero()

   -- next epoch
   epoch = epoch + 1
    time = time+1
    table.insert(times,time)
end


In [8]:
--classes={'bridge', 'building', 'city', 'eiffel_tower','elephant', 'landscape', 'lion', 'monkey', 'people', 'tower', 'water'}
totaltest = 0
for dir in paths.iterdirs("./dataset/val/") do
    files = {}
    print(files)
    for file in paths.files(paths.concat('./dataset/val/', dir)) do
       if file:find('JPEG' .. '$') then
          table.insert(files, 1)
       end
    end
   
    if #files == 0 then
       error('given directory doesnt contain any files of type: ')
    end
    print(totaltest)
    totaltest = totaltest + #files
    collectgarbage()
end

print('total', totaltest)

imagesAll = torch.Tensor(totaltest,3,64,64) 
labelsAll = torch.Tensor(totaltest)
compteur = 0
NumberOfImages = {0}
for dir in paths.iterdirs("./dataset/val/") do
    compteur = compteur +1
    files = {}
    for file in paths.files(paths.concat('./dataset/val/', dir)) do
       if file:find('JPEG' .. '$') then
          table.insert(files, paths.concat(paths.concat('./dataset/val/',dir), file))
       end
    end
   
    if #files == 0 then
       error('given directory doesnt contain any files of type: ')
    end

    lastElement = NumberOfImages[#NumberOfImages]
    table.insert(NumberOfImages, #files + lastElement)
    for i=1,(#files) do
        temp = image.load(files[i])
        temp2 = torch.Tensor(3,64,64) 
        if (temp:size()[1]==1) then
            temp2[{{1},{},{}}] = temp
            temp2[{{2},{},{}}] = temp
            temp2[{{3},{},{}}] = temp
        else 
            temp2 = temp
        end
        imagesAll[i+lastElement] = temp2 
        labelsAll[i+lastElement] = compteur
    end
    collectgarbage()
end


-- Nombre d'images courleurs: 1683 
testData = {
    data = torch.Tensor(totaltest, 3, 64, 64),
    labels = torch.Tensor(totaltest),
    size = function() return totaltest end,

}


for i=1,totaltest do
   testData.data[i] = imagesAll[i]
   testData.labels[i] = labelsAll[i]
end

-- Convert all images to YUV
print '==> preprocessing data: colorspace RGB -> YUV'
for i = 1,totaltest do
   testData.data[i] = image.rgb2yuv(testData.data[i])
end
--for i = 1,325 do
  -- testData.data[i] = image.rgb2yuv(testData.data[i])
--end
-- Name channels for convenience
channels = {'y','u','v'}

-- Normalize each channel, and store mean/std
-- per channel. These values are important, as they are part of
-- the trainable parameters. At test time, test data will be normalized
-- using these values.
print '==> preprocessing data: normalize each feature (channel) globally'
--mean = {}
--std = {}
for i,channel in ipairs(channels) do
   -- normalize each channel globally:
   testData.data[{ {},i,{},{} }]:add(-mean[i])
   testData.data[{ {},i,{},{} }]:div(std[i])
end

-- Local normalization
print '==> preprocessing data: normalize all three channels locally'

-- Define the normalization neighborhood:
neighborhood = image.gaussian1D(13)

-- Define our local normalization operator (It is an actual nn module, 
-- which could be inserted into a trainable model):
normalization = nn.SpatialContrastiveNormalization(1, neighborhood)

-- Normalize all channels locally:
for c in ipairs(channels) do

   for i = 1,totaltest do
      testData.data[{ i,{c},{},{} }] = normalization:forward(testData.data[{ i,{c},{},{} }])
   end
end

print '==> verify statistics'
for i,channel in ipairs(channels) do

   testMean = testData.data[{ {},i }]:mean()
   testStd = testData.data[{ {},i }]:std()


   print('test data, '..channel..'-channel, mean: ' .. testMean)
   print('test data, '..channel..'-channel, standard deviation: ' .. testStd)
end


{
}
0	


{
}
32	


{
}
64	
{
}
96	


{
}
107	


{
}
139	


...

==> preprocessing data: normalize all three channels locally	


==> verify statistics	


test data, y-channel, mean: -0.017539901252773	
test data, y-channel, standard deviation: 0.87259989006525	


test data, u-channel, mean: 0.018839901418081	
test data, u-channel, standard deviation: 0.86362135210153	


test data, v-channel, mean: -0.017297258674264	
test data, v-channel, standard deviation: 0.85897246440964	


In [9]:

-- test function
function test(dataset)

   -- test over given dataset
   print('<trainer> on testing Set:')
   for t = 1,dataset:size() do
      -- get new sample
      local input = dataset.data[t]
      local target = dataset.labels[t]

      -- test sample
      local pred = model:forward(input)
      confusion:add(pred, target)
   end

   -- print confusion matrix
   print(confusion)
    table.insert(evaluationsTest,confusion.averageValid)
   confusion:zero()

end


In [25]:
evaluationsTest={}
evaluations={}
i=0
while i<10 do
    train(trainData)
    test(testData)
    i=i+1
end
print(times)

<trainer> on training set:	
<trainer> online epoch # 21 [batchSize = 64]	


ConfusionMatrix:
[[     182       1       0       1       1       0       0       0       0       0       0]   98.378% 	[class: bridge]
 [       3     143       0       1       3       1       4       1       0       1       0]   91.083% 	[class: building]
 [       2       2      78       1       0       0       0       0       0       2       0]   91.765% 	[class: city]
 [       1       1       1     148       1       0       1       0       0       3       1]   94.268% 	[class: eiffel_tower]
 [       3       0       0       1     215       0       3       0       0       1       0]   96.413% 	[class: elephant]
 [       1       0       0       0       0      43       0       0       0       0       0]   97.727% 	[class: landscape]
 [       0       1       0       2       2       0     277       2       0       3       1]   96.181% 	[class: lion]
 [       2       1       0       0       0       0       4     204       1       2       1]   94.884% 	[class: monkey]
 [       0       0    

ConfusionMatrix:
[[       5       1       2       8       2       3       4       2       1       2       2]   15.625% 	[class: bridge]
 [       4       9       3       4       2       2       2       1       0       3       2]   28.125% 	[class: building]
 [       4       1       0       6       4       1       5       3       2       4       2]   0.000% 	[class: city]
 [       0       0       0      11       0       0       0       0       0       0       0]   100.000% 	[class: eiffel_tower]
 [       5       5       1       1       7       1       1       2       2       2       5]   21.875% 	[class: elephant]
 [       6       1       1       3       0       1       5       6       3       4       2]   3.125% 	[class: landscape]
 [       3       4       2       2       3       2       6       6       0       3       1]   18.750% 	[class: lion]
 [       5       2       2       3       2       1       3       6       3       1       4]   18.750% 	[class: monkey]
 [       3       3     

ConfusionMatrix:
[[     182       1       0       1       1       0       0       0       0       0       0]   98.378% 	[class: bridge]
 [       3     145       0       0       2       1       4       1       0       1       0]   92.357% 	[class: building]
 [       1       2      79       1       0       0       0       0       0       2       0]   92.941% 	[class: city]
 [       1       0       1     149       1       0       1       0       0       3       1]   94.904% 	[class: eiffel_tower]
 [       3       0       0       1     216       0       3       0       0       0       0]   96.861% 	[class: elephant]
 [       1       0       0       0       0      43       0       0       0       0       0]   97.727% 	[class: landscape]
 [       0       0       0       2       2       0     278       2       0       3       1]   96.528% 	[class: lion]
 [       2       0       0       0       0       0       4     206       0       2       1]   95.814% 	[class: monkey]
 [       0       0    

ConfusionMatrix:
[[       5       1       2       8       2       3       3       2       1       3       2]   15.625% 	[class: bridge]
 [       4       9       3       4       2       2       2       1       0       3       2]   28.125% 	[class: building]
 [       4       1       0       6       4       1       5       3       2       4       2]   0.000% 	[class: city]
 [       0       0       0      11       0       0       0       0       0       0       0]   100.000% 	[class: eiffel_tower]
 [       5       5       1       1       7       1       1       2       2       2       5]   21.875% 	[class: elephant]
 [       6       1       1       3       0       1       5       6       3       4       2]   3.125% 	[class: landscape]
 [       3       4       2       2       3       2       6       6       0       3       1]   18.750% 	[class: lion]
 [       5       2       2       3       2       1       3       6       2       2       4]   18.750% 	[class: monkey]
 [       3       3     

...

ConfusionMatrix:
[[     183       0       0       1       1       0       0       0       0       0       0]   98.919% 	[class: bridge]
 [       2     151       0       0       0       0       3       1       0       0       0]   96.178% 	[class: building]
 [       0       1      82       1       0       0       0       0       0       1       0]   96.471% 	[class: city]
 [       0       0       1     153       0       0       0       0       0       3       0]   97.452% 	[class: eiffel_tower]
 [       0       0       0       1     221       0       1       0       0       0       0]   99.103% 	[class: elephant]
 [       0       0       0       0       0      43       0       1       0       0       0]   97.727% 	[class: landscape]
 [       0       0       0       2       1       0     282       1       0       2       0]   97.917% 	[class: lion]
 [       2       0       0       0       0       0       4     207       0       1       1]   96.279% 	[class: monkey]
 [       0       0    

ConfusionMatrix:
[[       5       1       2       8       2       3       3       2       1       3       2]   15.625% 	[class: bridge]
 [       4       9       2       4       2       2       2       2       0       3       2]   28.125% 	[class: building]
 [       4       1       0       6       4       1       3       3       3       5       2]   0.000% 	[class: city]
 [       0       0       0      11       0       0       0       0       0       0       0]   100.000% 	[class: eiffel_tower]
 [       4       6       1       1       7       1       1       2       2       2       5]   21.875% 	[class: elephant]
 [       6       1       1       3       0       1       5       4       4       5       2]   3.125% 	[class: landscape]
 [       3       3       2       3       4       1       5       6       0       4       1]   15.625% 	[class: lion]
 [       5       2       2       3       2       1       3       6       2       2       4]   18.750% 	[class: monkey]
 [       4       2     

  valids : FloatTensor - size: 11
  mat : LongTensor - size: 11x11
  averageUnionValid : 0.089431722394445
  _targ_idx : LongTensor - empty
  averageValid : 0.21590909090909
  classes : 
    {
      1 : bridge
      2 : building
      3 : city
      4 : eiffel_tower
      5 : elephant
      6 : landscape
      7 : lion
      8 : monkey
      9 : people
      10 : tower
      11 : water
    }
  _prediction : FloatTensor - size: 11
  _pred_idx : LongTensor - size: 1
  nclasses : 11
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 11
  totalValid : 0.16616314199396
}
<trainer> on training set:	
<trainer> online epoch # 30 [batchSize = 64]	


ConfusionMatrix:
[[     183       0       0       1       1       0       0       0       0       0       0]   98.919% 	[class: bridge]
 [       2     151       0       0       0       0       3       1       0       0       0]   96.178% 	[class: building]
 [       0       1      83       0       0       0       0       0       0       1       0]   97.647% 	[class: city]
 [       0       0       1     153       0       0       0       0       0       3       0]   97.452% 	[class: eiffel_tower]
 [       0       0       0       1     221       0       1       0       0       0       0]   99.103% 	[class: elephant]
 [       0       0       0       0       0      44       0       0       0       0       0]   100.000% 	[class: landscape]
 [       0       0       0       1       1       0     283       1       0       2       0]   98.264% 	[class: lion]
 [       2       0       0       0       0       0       4     207       0       1       1]   96.279% 	[class: monkey]
 [       0       0   

ConfusionMatrix:
[[       5       1       2       8       2       3       3       2       1       3       2]   15.625% 	[class: bridge]
 [       4       9       2       4       2       2       2       2       0       3       2]   28.125% 	[class: building]
 [       4       1       0       6       4       1       2       3       4       5       2]   0.000% 	[class: city]
 [       0       0       0      11       0       0       0       0       0       0       0]   100.000% 	[class: eiffel_tower]
 [       4       6       1       1       7       1       1       2       2       2       5]   21.875% 	[class: elephant]
 [       7       1       1       2       0       1       5       4       4       5       2]   3.125% 	[class: landscape]
 [       3       3       2       3       4       1       5       6       0       4       1]   15.625% 	[class: lion]
 [       5       2       2       3       2       1       3       6       2       2       4]   18.750% 	[class: monkey]
 [       4       2     

In [26]:
for i=1,#evaluations do
  evaluations[i]=evaluations[i]*100
  evaluationsTest[i]=evaluationsTest[i]*100
end
print('TrainSet :',evaluations)
print('TestSet :',evaluationsTest)

TrainSet :	{
  1 : 94.813684983687
  2 : 95.624486966567
  3 : 96.187143434178
  4 : 96.592594818635
  5 : 96.894767067649
  6 : 97.101558338512
  7 : 97.201745076613
  8 : 97.647611661391
  9 : 97.856642441316
  10 : 98.261188918894
}
TestSet :	{
  1 : 21.306818181818
  2 : 21.306818181818
  3 : 21.306818181818
  4 : 21.306818181818
  5 : 21.306818181818
  6 : 21.590909090909
  7 : 21.590909090909
  8 : 21.590909090909
  9 : 21.590909090909
  10 : 21.875
}


In [28]:
require 'gnuplot'

--gnuplot.figure(1)
--gnuplot.title('SGD loss minimisation over time')
--gnuplot.plot(sgdtime, sgdevaluations)